# <b> 1. LSA(Latent Semantic Analysis)</b> 

In [1]:
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import TreebankWordTokenizer

In [2]:
df_2024 = pd.read_csv('../드라마영화예능_전처리데이터/24예능_전처리.csv')
df_2023 = pd.read_csv('../드라마영화예능_전처리데이터/23예능_전처리.csv')
df_2022 = pd.read_csv('../드라마영화예능_전처리데이터/22예능_전처리.csv')

df = pd.concat([df_2024, df_2023, df_2022], axis=0).reset_index(drop=True)

df.head()

,postdate,body,description,title,preprocessed_body,preprocessed_description,preprocessed_title
0,20240512,변우석 뻔우석 변신 런닝맨에서 펼쳐진 뻔뻔한 매력 분석 변우석의 뻔뻔한 매력이 런닝...,b예능b재미 b2024년b5월12일방송 b예능b프로그램 유재석제자 변우석재탄생 b예...,변우석 런닝맨에서 뻔우석 변신 SBS 예능 배고픔 뻔우석 식사 난입 가성비 꼼수 기...,변 우석 뻔우석 변신 런닝맨 매력 분석 변 우석 매력 런닝맨 전국 거실 글 런닝맨 ...,재미 유재석 제자 변 우석 탄생 신동 식사 변우 석 런닝맨 웃음 폭탄 대세 변우 석...,변 우석 런닝맨 뻔우 변신 배고픔 뻔우석 식사 난입 가성비 꼼수 기술 유재석 먹방 ...
1,20240510,안녕하세요 종합광고대행사 153프로덕션입니다 2024년 하반기에도 다채로운 소재의 ...,b2024년b 하반기에도 다채로운 소재의 b예능b이 방송될 예정입니다 기대되는 신규...,기대되는 신규예능 안내 MBC 장안의 화제SBS 정글밥 ENA 백종원의 레미제라블 ...,안녕하세요 광고 대행사 프로덕션 하반기 소재 장 화요일 밤 시 전국 로컬 마스터 장...,하반기 소재 장 화요일 밤 시 전국 로컬 마스터 장윤정,장안 정글 밥 백종원 레미제라블 사생활 현무카세 차트 쇼 언더 커버 간접광고 문
2,20240206,출처 넷플릭스 코리아 2024년 넷플릭스 분기별 한국 오리지널 드라마 영화 예능 라...,영화 b예능b 라인업 정보 넷플릭스가 지난주 금요일 b2024년b 공개 예정 메인 ...,2024년 넷플릭스 분기별 한국 오리지널 드라마 영화 예능 라인업 정보,넷플릭스 코리아 넷플릭스 분기 넷플릭스 지나 주 금요일 메인 소개 올 해 분기 별 ...,넷플릭스 지나 주 금요일 메인 소개 자유 의미,넷플릭스 분기 별
3,20240418,때는 2024년 대유튜브의 시대를 지나 대OTT의 시대가 도래했으니 영화 드라마 예...,때는 b2024년b 대유튜브의 시대를 지나 대OTT의 시대가 도래했으니 영화 드라마...,서바이벌 예능에서 연프 찍는다고 화제된 출연자jpg,유튜브 시대 지나 대 시대 도래 와중 세계 거대 회사 넷플릭스 발표 인 탈출 말 게...,유튜브 시대 지나 대 시대 도래 순식간 이목 집중 작년 넷플릭스 강철 부대 우승,서바이벌 연프 출연자
4,20240510,5월엔 새로운 예능이 정말 많은 것 같아요 대한민국 구석구석을 방문하는 예능이 많이...,5월엔 새로운 b예능b이 정말 많은 것 같아요 대한민국 구석구석을 방문하는 b예능b...,엘크로 2024년 5월 신규 예능 라인업 세 번째,대한민국 구석구석 방문 번 소개 전국 팔도 로컬 마스터 장윤정 안정환 팝업 스토어 ...,대한민국 구석구석 방문 레전드 인생 순간 노래 노래,엘크 번


In [3]:
# 결측치 제거
df = df.dropna(subset=['preprocessed_body'])
df.shape

(1709, 7)

In [4]:
# 상위 (max_features)개의 단어에 대해서만 vectorize  -> 각각의 문서들을 벡터 형태로 만들기 위해서 사용
# max_df=0.5: 문서의 절반 이상에서 나타나는 단어를 무시 -> 자주 등장하지만 정보량이 적은 단어 
vectorizer = TfidfVectorizer(max_features=300, max_df=0.5, smooth_idf=True) 
X = vectorizer.fit_transform(df['preprocessed_body'])

In [5]:
print(X.shape)

(1709, 300)


In [6]:
# X는 sparse matrix. 
# 효율적인 저장 방식인 CSR(Compressed Sparse Row) matrix 형식으로 저장되어 있음
X

<1709x300 sparse matrix of type '<class 'numpy.float64'>'
	with 64801 stored elements in Compressed Sparse Row format>

- <span style = 'font-size:1.1em;line-height:1.5em'>TF-IDF의 각 차원의 의미</span>

In [7]:
terms = vectorizer.get_feature_names_out()
print(terms) # 벡터의 각 차원이 어떤 단어를 의미하는지 
print(len(terms)) # 총 벡터의 차원은 300차원 

['가능' '가요' '가운데' '가족' '가지' '감독' '감동' '감사' '감정' '개봉' '개월' '개인' '걸그룹' '게스트'
 '게임' '결과' '결정' '결혼' '결혼식' '경기' '경우' '경험' '고민' '공감' '공식' '공연' '과거' '과정'
 '관객' '관계' '관련' '광고' '구성' '국가' '국내' '국민' '그룹' '글로벌' '기대' '기록' '기사' '기안'
 '기억' '기자' '기존' '기회' '기획' '김호중' '나라' '남매' '남편' '내용' '네이버' '넷플릭스' '노래' '노력'
 '논란' '눈물' '뉴스' '느낌' '능력' '대상' '대중' '대학' '대한민국' '댄스' '데이트' '덱스' '도전' '동시'
 '동영상' '디즈니' '라디오' '라면' '라이브' '런닝맨' '로맨스' '리얼리티' '마지막' '맛집' '매력' '메인' '모델'
 '무대' '문제' '문화' '미국' '반응' '발매' '발표' '방식' '방영' '변화' '보컬' '본인' '봄날' '부모'
 '부문' '부부' '부분' '분석' '분야' '분위기' '브랜드' '사건' '사고' '사실' '사용' '사이' '사회' '상대'
 '상황' '생활' '서바이벌' '서비스' '서울' '선사' '선수' '선택' '성공' '성장' '세계' '세대' '세상' '소개'
 '소속' '소속사' '소재' '솔로' '수상' '순간' '순위' '스타' '스토리' '스포츠' '시대' '시리즈' '시상식'
 '시장' '시절' '시점' '시청자' '실력' '아나운서' '아내' '아들' '아버지' '아빠' '아이돌' '아티스트' '앨범'
 '야구' '어머니' '언급' '언니' '얼굴' '엄마' '에피소드' '엔터테인먼트' '여성' '여자' '여행' '역대' '역할'
 '연애' '연예' '연예인' '연인' '연출' '열애' '예고' '예고편' '예상' '예술' '오늘' '오디션' '오후' '온라인'
 '올해' '외모' '요리' '요즘' '우석' '우승' '운영' '웃음' '원작' '웨

- <span style = 'font-size:1.1em;line-height:1.5em'>TF-IDF matrix에 Truncated SVD를 적용하여 LSA 수행</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>n_components = 토픽 수</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>일반적으로 TF-IDF 특징 수의 5%에서 20% 사이의 값을 토픽 수로 설정

In [8]:
model = TruncatedSVD(n_components=30, n_iter=100, random_state=42) 
model.fit(X)

TruncatedSVD(n_components=30, n_iter=100, random_state=42)

$$X=U \Sigma V^{T} \approx U_{trunc} \Sigma_{trunc} V_{trunc}^{T}$$ 
- <span style = 'font-size:1.2em;line-height:1.5em'>여기서 <code>result=model.transform(X)</code>: $U_{trunc} \Sigma_{trunc}$</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>1709개의 문서를 300차원의 벡터로 (즉, 300개 단어의 조합으로) 표현</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>그러나, SVD를 활용하여 1709개의 문서를 30차원의 벡터로 (즉, 10개 주제의 조합으로) 표현 </span>

In [9]:
result = model.transform(X)
print(result.shape)

(1709, 30)


In [10]:
result.shape

(1709, 30)

In [11]:
# 30개의 각 topic이 어떻게 300개의 단어 조합으로 되어있는지
print(model.components_.shape)
print(model.components_)

(30, 300)
[[ 0.06466108  0.03564099  0.03984353 ...  0.04072989  0.03424297
   0.03665327]
 [-0.01455106 -0.02696901 -0.00302917 ...  0.00372378 -0.01927686
  -0.01035007]
 [-0.01931894  0.00368777  0.00101076 ... -0.00329025 -0.02055439
  -0.03124035]
 ...
 [-0.03770778 -0.01786551 -0.00772835 ... -0.0091624  -0.06917431
   0.02611353]
 [ 0.00103181  0.0244472  -0.01368975 ... -0.02809564  0.0902021
  -0.03977884]
 [ 0.02046208  0.00385527 -0.02365517 ...  0.03389834 -0.00410197
   0.00337628]]


In [12]:
terms = vectorizer.get_feature_names_out()
print(terms) # 벡터의 각 차원이 어떤 단어를 의미하는지 (Each dimension is matched with which word)
print(len(terms)) # 총 벡터의 차원은 300차원 (Dimension: 300)

['가능' '가요' '가운데' '가족' '가지' '감독' '감동' '감사' '감정' '개봉' '개월' '개인' '걸그룹' '게스트'
 '게임' '결과' '결정' '결혼' '결혼식' '경기' '경우' '경험' '고민' '공감' '공식' '공연' '과거' '과정'
 '관객' '관계' '관련' '광고' '구성' '국가' '국내' '국민' '그룹' '글로벌' '기대' '기록' '기사' '기안'
 '기억' '기자' '기존' '기회' '기획' '김호중' '나라' '남매' '남편' '내용' '네이버' '넷플릭스' '노래' '노력'
 '논란' '눈물' '뉴스' '느낌' '능력' '대상' '대중' '대학' '대한민국' '댄스' '데이트' '덱스' '도전' '동시'
 '동영상' '디즈니' '라디오' '라면' '라이브' '런닝맨' '로맨스' '리얼리티' '마지막' '맛집' '매력' '메인' '모델'
 '무대' '문제' '문화' '미국' '반응' '발매' '발표' '방식' '방영' '변화' '보컬' '본인' '봄날' '부모'
 '부문' '부부' '부분' '분석' '분야' '분위기' '브랜드' '사건' '사고' '사실' '사용' '사이' '사회' '상대'
 '상황' '생활' '서바이벌' '서비스' '서울' '선사' '선수' '선택' '성공' '성장' '세계' '세대' '세상' '소개'
 '소속' '소속사' '소재' '솔로' '수상' '순간' '순위' '스타' '스토리' '스포츠' '시대' '시리즈' '시상식'
 '시장' '시절' '시점' '시청자' '실력' '아나운서' '아내' '아들' '아버지' '아빠' '아이돌' '아티스트' '앨범'
 '야구' '어머니' '언급' '언니' '얼굴' '엄마' '에피소드' '엔터테인먼트' '여성' '여자' '여행' '역대' '역할'
 '연애' '연예' '연예인' '연인' '연출' '열애' '예고' '예고편' '예상' '예술' '오늘' '오디션' '오후' '온라인'
 '올해' '외모' '요리' '요즘' '우석' '우승' '운영' '웃음' '원작' '웨

In [13]:
model.components_[0].shape

(300,)

- <span style = 'font-size:1.2em;line-height:1.5em'>각 topic에 대해서 상위 5개의 큰 값을 갖는 단어를 출력하는 함수</span>

In [14]:
def get_keyword_by_topic(components, feature_name, n=5):
    for idx, topic in enumerate(components):
        sorted_keyword_idx = np.argsort(components[idx])[-1:-n-1:-1]
        sorted_keyword_result = [(feature_name[x], topic[x].round(4)) for x in sorted_keyword_idx]
        print(f'Topic {idx}: {sorted_keyword_result}')

In [15]:
get_keyword_by_topic(model.components_, terms)

Topic 0: [('연애', 0.1962), ('결혼', 0.1811), ('넷플릭스', 0.1657), ('게임', 0.1219), ('유튜브', 0.1198)]
Topic 1: [('연애', 0.6993), ('환승', 0.3528), ('티빙', 0.2479), ('커플', 0.185), ('솔로', 0.1401)]
Topic 2: [('결혼', 0.5808), ('이혼', 0.3934), ('남편', 0.1888), ('부부', 0.1696), ('결혼식', 0.1551)]
Topic 3: [('넷플릭스', 0.5592), ('지옥', 0.3194), ('솔로', 0.2324), ('결혼', 0.223), ('이혼', 0.1812)]
Topic 4: [('티빙', 0.4465), ('여행', 0.3079), ('이혼', 0.249), ('추리', 0.1031), ('지구', 0.0905)]
Topic 5: [('음악', 0.3789), ('노래', 0.2541), ('티빙', 0.2221), ('이혼', 0.1946), ('무대', 0.1939)]
Topic 6: [('선수', 0.2856), ('축구', 0.2625), ('야구', 0.2046), ('경기', 0.1766), ('일본', 0.1629)]
Topic 7: [('티빙', 0.4066), ('선수', 0.2791), ('야구', 0.2474), ('축구', 0.2264), ('유재석', 0.1959)]
Topic 8: [('여행', 0.6851), ('광고', 0.1144), ('솔로', 0.1083), ('음식', 0.0946), ('졸업', 0.0853)]
Topic 9: [('야구', 0.3551), ('최강', 0.2282), ('선수', 0.2184), ('이혼', 0.2016), ('경기', 0.1468)]
Topic 10: [('전현무', 0.2803), ('연예', 0.2781), ('대상', 0.2501), ('열애', 0.1994), ('여행', 0.1978)]
Topi

# <b> 2. LDA(Latent Dirichlet Allocation)</b> 

In [16]:
# preprocessed_body에 있는 단어를 리스트에 넣은 칼럼 생성
df['word_list'] = df['preprocessed_body'].apply(lambda x: x.split())

In [17]:
df['word_list'].value_counts()

word_list
[본문, 수]                                                                                                                                                                                                                                                                                                                                                                                                                                                       10
[변, 우석, 뻔우석, 변신, 런닝맨, 매력, 분석, 변, 우석, 매력, 런닝맨, 전국, 거실, 글, 런닝맨, 변, 우석, 뻔우, 석, 탄생, 시청자, 웃음, 선사, 매력, 조명, 에피소드, 이색, 재미, 창출, 런닝맨, 명장면, 탄생, 변, 우석, 번, 런닝, 달, 초보, 시절, 변, 우석, 씨, 번, 런닝맨, 접수, 경이, 성, 비, 탄, 레이스, 여정, 가성비, 식당, 탐방, 가격, 추리, 게임, 치열, 식사, 기회, 고군분투, 사이, 우석, 씨, 꼴찌, 소리, 연발, 뻔, 우석, 탄생, 비화, 배고픔, 변신, 식사, 기회, 아우성, 서, 유재석, 속도, 생명, 입, 지혜, 전수, 변, 우석, 씨, 말, 심장, 음식, 앞, 지, 수, 탄생, 뻔우석, 배고픔, 거, 식사, 난입, 현장, 변, ...]                                                 1
[작년, 여유, 오역, 점, 것, 번, 인터뷰, 날, 마련, 임, 스튜디오, 곁, 산들바람, 장미, 향기, 듯, 얼굴, 평온, 편안, 해, 데, 학기, 말, 결과, 

In [18]:
lst = df['word_list'].apply(lambda words: [word for word in words if len(word) == 1])
lst
# word_list에서 한 글자인 단어 확인 -> 무의미하다고 판단

0       [변, 변, 글, 변, 석, 변, 번, 달, 변, 씨, 번, 성, 비, 탄, 씨, ...
1       [장, 밤, 시, 개, 중, 장, 개, 중, 퀸, 티, 엔, 핫, 톱, 몸, 서, ...
2       [주, 올, 해, 별, 컷, 수, 도, 적, 해, 수, 작, 날, 딸, 빠, 극, ...
3       [대, 인, 말, 중, 인, 달, 수, 주, 니, 도, 명, 장, 중, 세, 김, ...
4       [번, 개, 중, 장, 힙, 지, 화, 금, 시, 키, 시, 분, 곳, 단, 다, ...
                              ...                        
1704    [유, 온, 더, 분, 번, 설, 수, 역, 씨, 데, 씬, 씬, 대, 씬, 유, ...
1705    [토, 해, 분, 멋, 중, 거, 번, 시, 님, 외, 한, 분, 님, 개, 님, ...
1706    [배, 시, 분, 배, 배, 지, 배, 라, 배, 개, 곳, 금, 빵, 립, 도, ...
1707    [되, 데, 말, 면, 곡, 룸, 알, 젓, 밤, 시, 랩, 랩, 것, 랩, 지, ...
1708    [집, 집, 딸, 윤, 대, 라, 데, 살, 맨, 라, 김, 라, 중, 꼴, 바, ...
Name: word_list, Length: 1709, dtype: object

In [19]:
# word_list에서 한 글자인 단어 삭제
df['word_list'] = df['word_list'].apply(lambda words: [word for word in words if len(word) > 1])
df['word_list'].value_counts()

word_list
[본문]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    10
[우석, 뻔우석, 변신, 런닝맨, 매력, 분석, 우석, 매력, 런닝맨, 전국, 거실, 런닝맨, 우석, 뻔우, 탄생, 시청자, 웃음, 선사, 매력, 조명, 에피소드, 이색, 재미, 창출, 런닝맨, 명장면, 탄생, 우석, 런닝, 초보, 시절, 우석, 런닝맨, 접수, 경이, 레이스, 여정, 가성비, 식당, 탐방, 가격, 추리, 게임, 치열, 식사, 기회, 고군분투, 사이, 우석, 꼴찌, 소리, 연발, 우석, 탄생, 비화, 배고픔, 변신, 식사, 기회, 아우성, 유재석, 속도, 생명, 지혜, 전수, 우석, 심장, 음식, 탄생, 뻔우석, 배고픔, 식사, 난입, 현장, 우석, 배고픔, 식사, 자리, 반칙, 기술, 웃음바다, 행동, 현장, 웃음, 특별, 확대, 편성, 우석, 오늘, 런닝맨, 특별, 확대, 편성, 우석, 우석, 면모, 예측, 불허, 에피소드, 결론, ...]                                           1
[작년, 여유, 오역, 인터뷰, 마련, 스튜디오, 산들바람, 장미, 향기

In [20]:
from gensim import corpora

word_dict = corpora.Dictionary(df['word_list'])

In [21]:
corpus = [word_dict.doc2bow(text) for text in df['word_list']]

In [22]:
len(corpus)

1709

In [23]:
corpus[2]  ## 2번째 문서에 대해 각각의 단어들이 몇 번을 나왔는지.. 
## -> (1, 1): 1번 단어가 1번 나오고... 
## -> (4, 3): 4번 단어가 3번 나오고...

[(1, 1),
 (4, 3),
 (7, 2),
 (46, 1),
 (64, 2),
 (67, 1),
 (78, 1),
 (83, 1),
 (90, 1),
 (111, 2),
 (125, 1),
 (141, 1),
 (145, 5),
 (162, 1),
 (197, 6),
 (205, 2),
 (250, 1),
 (252, 2),
 (256, 1),
 (259, 2),
 (290, 7),
 (328, 1),
 (329, 1),
 (330, 1),
 (331, 2),
 (332, 1),
 (333, 1),
 (334, 4),
 (335, 2),
 (336, 3),
 (337, 1),
 (338, 2),
 (339, 1),
 (340, 3),
 (341, 1),
 (342, 2),
 (343, 1),
 (344, 2),
 (345, 1),
 (346, 1),
 (347, 1),
 (348, 1),
 (349, 1),
 (350, 1),
 (351, 1),
 (352, 1),
 (353, 2),
 (354, 1),
 (355, 1),
 (356, 1),
 (357, 2),
 (358, 2),
 (359, 1),
 (360, 1),
 (361, 1),
 (362, 1),
 (363, 1),
 (364, 1),
 (365, 1),
 (366, 2),
 (367, 1),
 (368, 1),
 (369, 1),
 (370, 1),
 (371, 1),
 (372, 1),
 (373, 1),
 (374, 2),
 (375, 1),
 (376, 1),
 (377, 2),
 (378, 1),
 (379, 26),
 (380, 1),
 (381, 1),
 (382, 2),
 (383, 1),
 (384, 1),
 (385, 1),
 (386, 1),
 (387, 1),
 (388, 2),
 (389, 1),
 (390, 1),
 (391, 2),
 (392, 1),
 (393, 1),
 (394, 1),
 (395, 2),
 (396, 1),
 (397, 1),
 (398, 1),

In [24]:
[(idx,word) for idx, word in word_dict.items()]
## 0번 단어는 가족...
## 1번 단어는 가지...

[(0, '가격'),
 (1, '가능'),
 (2, '가성비'),
 (3, '거실'),
 (4, '게임'),
 (5, '결론'),
 (6, '경이'),
 (7, '고군분투'),
 (8, '그림'),
 (9, '기대감'),
 (10, '기술'),
 (11, '기존'),
 (12, '기회'),
 (13, '꼴찌'),
 (14, '난입'),
 (15, '대세'),
 (16, '등장'),
 (17, '런닝'),
 (18, '런닝맨'),
 (19, '레이스'),
 (20, '매력'),
 (21, '먹방'),
 (22, '면모'),
 (23, '명장면'),
 (24, '반칙'),
 (25, '배고픔'),
 (26, '변신'),
 (27, '변우'),
 (28, '별명'),
 (29, '분석'),
 (30, '불허'),
 (31, '비화'),
 (32, '뻔우'),
 (33, '뻔우석'),
 (34, '사이'),
 (35, '생명'),
 (36, '석반'),
 (37, '선사'),
 (38, '소리'),
 (39, '속도'),
 (40, '수행'),
 (41, '시절'),
 (42, '시청자'),
 (43, '식당'),
 (44, '식사'),
 (45, '신동'),
 (46, '신선'),
 (47, '심장'),
 (48, '아우성'),
 (49, '에피소드'),
 (50, '여정'),
 (51, '역할'),
 (52, '연발'),
 (53, '예상'),
 (54, '예측'),
 (55, '오늘'),
 (56, '요소'),
 (57, '우석'),
 (58, '웃음'),
 (59, '웃음바다'),
 (60, '유재석'),
 (61, '음식'),
 (62, '이벤트'),
 (63, '이색'),
 (64, '인물'),
 (65, '자리'),
 (66, '자리매김'),
 (67, '재미'),
 (68, '전국'),
 (69, '전수'),
 (70, '접수'),
 (71, '제자'),
 (72, '조명'),
 (73, '중심'),
 (74, '지혜'),
 (75, '창출'),
 (7

## LDA 모델 훈련 (Topic 수: 30개로 설정)

In [25]:
import gensim

In [26]:
N_TOPICS = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics = N_TOPICS,  ## 주제 몇 개 할 건지
                                           id2word=word_dict)   ## vocabulary set

### 각 topic의 대표 단어들을 출력

In [27]:
topics = ldamodel.print_topics(num_words=5)  ## 각 토픽들에 대해서 몇 개의 단어(num_words)로 표현할 것인지
## 대표적인 단어를 알 수 있음
for topic in topics:
    print(topic)

(13, '0.006*"결혼" + 0.004*"연애" + 0.003*"이효리" + 0.003*"음악" + 0.002*"유튜브"')
(11, '0.004*"솔로" + 0.004*"결혼" + 0.003*"지옥" + 0.003*"넷플릭스" + 0.003*"연애"')
(28, '0.007*"결혼" + 0.003*"이혼" + 0.003*"축구" + 0.003*"가족" + 0.003*"넷플릭스"')
(16, '0.003*"게임" + 0.003*"넷플릭스" + 0.003*"스타" + 0.003*"선수" + 0.003*"음악"')
(20, '0.006*"연애" + 0.005*"남매" + 0.004*"음악" + 0.004*"게임" + 0.003*"결혼"')
(29, '0.009*"연애" + 0.006*"결혼" + 0.003*"신병" + 0.003*"선수" + 0.002*"넷플릭스"')
(2, '0.007*"넷플릭스" + 0.005*"티빙" + 0.004*"서울" + 0.003*"결혼" + 0.003*"걸그룹"')
(26, '0.005*"넷플릭스" + 0.003*"결혼" + 0.003*"게임" + 0.003*"음악" + 0.002*"연애"')
(23, '0.004*"스타" + 0.003*"연예" + 0.003*"세계" + 0.003*"여행" + 0.002*"넷플릭스"')
(15, '0.004*"추리" + 0.004*"넷플릭스" + 0.003*"송범" + 0.003*"이미주" + 0.003*"정원"')
(5, '0.005*"신병" + 0.005*"티빙" + 0.004*"부대" + 0.004*"맛집" + 0.003*"야구"')
(4, '0.004*"여행" + 0.003*"중국" + 0.003*"스타" + 0.003*"대한민국" + 0.003*"음악"')
(17, '0.007*"연애" + 0.004*"직업" + 0.004*"환승" + 0.003*"커플" + 0.003*"여자"')
(22, '0.003*"대한민국" + 0.003*"여자" + 0.002*"스타" + 0.002*"중국" 

## LDA 시각화

In [28]:
from IPython.core.display import HTML
import pyLDAvis
from pyLDAvis import gensim_models

In [29]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, word_dict)
pyLDAvis.display(vis)

## 문서 별 Topic 분포

In [30]:
for i, topic_list in enumerate(ldamodel[corpus]):
    print(f'{i}번째 문서의 topic 비중: {topic_list}')
    if i==5:
        break

0번째 문서의 topic 비중: [(2, 0.99433905)]
1번째 문서의 topic 비중: [(5, 0.9973894)]
2번째 문서의 topic 비중: [(6, 0.35814348), (7, 0.079425246), (10, 0.2786847), (21, 0.2821906)]
3번째 문서의 topic 비중: [(10, 0.121709175), (12, 0.21321341), (14, 0.06330326), (27, 0.03210206), (29, 0.56889516)]
4번째 문서의 topic 비중: [(4, 0.10460685), (5, 0.28743002), (27, 0.5995318)]
5번째 문서의 topic 비중: [(0, 0.9973072)]


### 각 문서에 대해서 Topic의 비중을 나타내는 함수 작성

In [31]:
## 교수님 코드 

def get_topic_ratio_for_each_document(ldamodel, corpus):
    results = []
    for i, topic_list in enumerate(ldamodel[corpus]):
        sorted_topic_list = sorted(topic_list, key = lambda x: x[1], reverse=True)
        most_important_topic, most_important_ratio = sorted_topic_list[0]
        doc_result = [i, most_important_topic, most_important_ratio, sorted_topic_list]
        results.append(doc_result)
        
    results = pd.DataFrame(results, columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중'])
    return results

In [32]:
df_results = get_topic_ratio_for_each_document(ldamodel, corpus)

df_results

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,2,0.994339,"[(2, 0.99433905)]"
1,1,5,0.997389,"[(5, 0.9973894)]"
2,2,6,0.356501,"[(6, 0.35650146), (21, 0.28204983), (10, 0.281..."
3,3,29,0.569011,"[(29, 0.56901103), (12, 0.21326698), (10, 0.12..."
4,4,27,0.600641,"[(27, 0.60064125), (5, 0.28768256), (4, 0.1032..."
...,...,...,...,...
1704,1704,16,0.992251,"[(16, 0.9922511)]"
1705,1705,8,0.996032,"[(8, 0.9960323)]"
1706,1706,20,0.992186,"[(20, 0.99218625)]"
1707,1707,20,0.759059,"[(20, 0.7590588), (16, 0.23699695)]"


In [33]:
## 참고 사이트: https://wikidocs.net/30708

def make_topictable_per_doc(ldamodel, corpus):
    # 데이터를 저장할 빈 리스트 초기화
    topic_data = []

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_data.append([i, int(topic_num), round(prop_topic, 4), topic_list])
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
            
    # 리스트를 데이터프레임으로 변환
    topic_table = pd.DataFrame(topic_data, columns=['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중'])

    return(topic_table)

In [34]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable[:10]

,index,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,0,2,0.9943,"[(2, 0.99433905)]"
1,1,1,5,0.9974,"[(5, 0.9973894)]"
2,2,2,6,0.3571,"[(6, 0.35706207), (7, 0.07514857), (10, 0.2794..."
3,3,3,29,0.5684,"[(10, 0.12072697), (12, 0.21267208), (14, 0.06..."
4,4,4,27,0.6189,"[(4, 0.08076544), (5, 0.29192692), (27, 0.6188..."
5,5,5,0,0.9973,"[(0, 0.9973072)]"
6,6,6,1,0.9974,"[(1, 0.99739903)]"
7,7,7,18,0.9853,"[(18, 0.98534423)]"
8,8,8,7,0.5870,"[(7, 0.58701444), (18, 0.40334937)]"
9,9,9,20,0.7083,"[(15, 0.011322916), (20, 0.70834774), (25, 0.2..."


In [35]:
## 교수님 코드 + 각 토픽 별 키워드를 볼 수 있는 함수 (GPT의 도움을 조금 받음!)


def get_topic_keywords(ldamodel, num_keywords=5):
    topic_keywords = {}
    for topic_id in range(ldamodel.num_topics):
        keywords = [word for word, _ in ldamodel.show_topic(topic_id, topn=num_keywords)]
        topic_keywords[topic_id] = keywords
    return topic_keywords

def get_topic_ratio_for_each_document(ldamodel, corpus):
    topic_keywords = get_topic_keywords(ldamodel)
    results = []
    for i, topic_list in enumerate(ldamodel[corpus]):
        sorted_topic_list = sorted(topic_list, key = lambda x: x[1], reverse=True)
        most_important_topic, most_important_ratio = sorted_topic_list[0]
        second_most_important_topic, second_most_important_ratio = sorted_topic_list[1] if len(sorted_topic_list) > 1 else (None, 0)

        keywords_1 = ", ".join(topic_keywords[most_important_topic])
        keywords_2 = ", ".join(topic_keywords[second_most_important_topic]) if second_most_important_topic is not None else ''
        doc_result = [i, most_important_topic, most_important_ratio, sorted_topic_list, keywords_1, keywords_2]
        results.append(doc_result)
        
    results = pd.DataFrame(results, columns=['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중', 'Top1_Topic_keyword', 'Top2_Topic_keyword'])
    return results

In [36]:
df_results = get_topic_ratio_for_each_document(ldamodel, corpus)

df_results

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중,Top1_Topic_keyword,Top2_Topic_keyword
0,0,2,0.994339,"[(2, 0.99433905)]","넷플릭스, 티빙, 서울, 결혼, 걸그룹",
1,1,5,0.997389,"[(5, 0.9973894)]","신병, 티빙, 부대, 맛집, 야구",
2,2,6,0.358140,"[(6, 0.35814044), (21, 0.28365615), (10, 0.280...","넷플릭스, 게임, 음악, 유튜브, 덱스","티빙, 게임, 넷플릭스, 연애, 유튜브"
3,3,29,0.569362,"[(29, 0.56936246), (12, 0.214502), (10, 0.1222...","연애, 결혼, 신병, 선수, 넷플릭스","연애, 대한민국, 티빙, 음악, 게임"
4,4,27,0.602172,"[(27, 0.6021719), (5, 0.2880157), (4, 0.1013811)]","연애, 넷플릭스, 스타, 남매, 결혼","신병, 티빙, 부대, 맛집, 야구"
...,...,...,...,...,...,...
1704,1704,16,0.992251,"[(16, 0.9922511)]","게임, 넷플릭스, 스타, 선수, 음악",
1705,1705,8,0.996032,"[(8, 0.9960323)]","연예, 스타, 넷플릭스, 중국, 시장",
1706,1706,20,0.992186,"[(20, 0.99218625)]","연애, 남매, 음악, 게임, 결혼",
1707,1707,20,0.757724,"[(20, 0.7577238), (16, 0.23813225)]","연애, 남매, 음악, 게임, 결혼","게임, 넷플릭스, 스타, 선수, 음악"
